# 2024.11.13.수 day13

## 1. 다나와 크롤링

### 셀레늄, 크롬드라이버 사용

In [117]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.common.by import By
# from bs4 import BeautifulSoup as BS
import requests
import time
from bs4 import BeautifulSoup as BS

options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager(driver_version="131.0.6778.69").install()),options=options)

url = "https://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2024-09-00&MonthTo="
driver.get(url)

bs = BS(driver.page_source)

In [123]:
# bs

### 2022.01 ~ 2024.10 판매실적 데이터 크롤링

In [176]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
from bs4 import BeautifulSoup as BS
import pandas as pd
from tqdm import tqdm

options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager(driver_version="131.0.6778.69").install()),options=options)

url_base = "https://auto.danawa.com/auto/?Work=record&Tab=Model&Month={}-{}-00&MonthTo="

li_sale = list()

# 연도, 월 설정하는 반복문
for year in ['2022', '2023', '2024']:
    for month in tqdm(['{:02}'.format(x) for x in range(1,13)]):
        url = url_base.format(year, month)
        driver.get(url)
        time.sleep(3)
        bs = BS(driver.page_source)
        
        if bs.find('tbody') == None:
            break
        
        li_tr = bs.find('tbody').findAll('tr')
        li_tr_new = [tr for tr in li_tr if not tr.get('class')]
        
        for t in li_tr_new:
            rank = t.findAll('td')[1].text
            car_name = t.findAll('td')[3].text.strip()
            val_sale = t.findAll('td')[4].text.split('그')[0]
            per_share = t.findAll('td')[5].text
            cd_brand = t.find('input')['brand']
            cd_model = t.find('input')['value'].split('_')[1]
            li_sale.append([year, month, rank, car_name, cd_model, cd_brand, val_sale, per_share])

df_sale = pd.DataFrame(li_sale, columns= ['year','month','rank','car_name','cd_model','cd_brand','val_sale','per_share'])

 83%|██████████████████████████████▊      | 10/12 [00:49<00:09,  4.91s/it]


In [1]:
# df_sale.info()

In [2]:
# df_sale

### 데이터 전처리

In [219]:
# 컬럼 타입 변경
df_sale_p = df_sale.copy()
df_sale_p['val_sale'] = df_sale_p['val_sale'].apply(lambda x: int(x.replace(',','')))
# df_sale_p['per_share'] = df_sale_p['per_share'].apply(lambda x: float(x.replace('%',''))/100)

# 월별 전체 판매량
df_group = df_sale_p.groupby(['year','month'])['val_sale'].sum().reset_index()
df_group = df_group.rename(columns={'val_sale':'total_sale'})

# merge 병합
df_sale_p2 = pd.merge(df_sale_p, df_group, how='left', on=['year','month'])
df_sale_p2['per_share2'] = df_sale_p2['val_sale'] / df_sale_p2['total_sale']
df_sale_p2

,year,month,rank,car_name,cd_model,cd_brand,val_sale,per_share,total_sale,per_share2
0,2022,01,1,G80,3656,304,5501,0.050,110802,0.049647
1,2022,01,2,포터2,1901,303,5443,0.049,110802,0.049124
2,2022,01,3,아반떼,3821,303,5099,0.046,110802,0.046019
3,2022,01,4,쏘렌토,3837,307,5066,0.046,110802,0.045721
4,2022,01,5,봉고 3,3772,307,4643,0.042,110802,0.041904
...,...,...,...,...,...,...,...,...,...,...
7807,2024,10,233,A-Class,3691,349,1,0.000,145281,0.000007
7808,2024,10,234,Q8 e-tron,4436,371,1,0.000,145281,0.000007
7809,2024,10,235,GLE-Class,3693,349,1,0.000,145281,0.000007
7810,2024,10,236,CT5,3709,546,1,0.000,145281,0.000007


In [3]:
# df_sale_p2.info()

### DB에 테이블로 저장

In [230]:
# DB 연결
import sqlalchemy
from urllib import parse
user='team2'
password='Encore_team2@'
password = parse.quote_plus(password)
host='192.168.0.95'
#host='222.112.208.67'
port=3306
database='team2'
engine=sqlalchemy.create_engine(f'mysql://{user}:{password}@{host}:{port}/{database}')

In [231]:
# df_sale_p2.to_sql('danawa_sale', if_exists='replace', con=engine, index=False)

7812